In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import emcee
import sys
from multiprocessing import Pool

import os

os.environ["OMP_NUM_THREADS"] = "1"

In [ ]:
# Dados começam 06/03/2020
# Vacinação começa 19/01/2022
dados = pd.read_csv("./generated_data/cleanned_data.csv").values

dados_datas = dados[:,0][:311]
dados_mortes = dados[:,1][:311]
dados_casos = dados[:,2][:311]
dados_hospitalizados = dados[:,3][:311]
dados_infectados_vacinados = dados[:,4][:311]
dados_vacinados_acumulados = dados[:,5][:311]
dados_hospitalizados_vacinados = dados[:,6][:311]
Psi = dados[:,7][:311]
N=14136417

In [ ]:
# evitando valores negativos nas vacinações diárias

vacinados_dados_aux =  np.copy(dados_vacinados_acumulados)
# Diffs menores que 0 serão substituidos pela repetição do valor anterior, tornando todo valor igual ou maior que o anterior
for i in range(1,len(vacinados_dados_aux)):
    diff = vacinados_dados_aux[i] - vacinados_dados_aux[i-1]
    if diff < 0:
        vacinados_dados_aux[i-1] = vacinados_dados_aux[i]

novos_vacinados =  np.diff(vacinados_dados_aux)
novos_vacinados = novos_vacinados.tolist()
novos_vacinados.insert(0, 0)

In [ ]:
# Parametros do modelo
# Ultima revisão: 05/01/2023
k = 1/4
gamma_a = 1/3.5
gamma_s = 1/4
gamma_h = 0.18
gamma_u = 0.13
mi_u = 0.4
qsi = 0.53 
mi_h = 0.15
ome_h = 0.14
ome_u = 0.29
p = 0.2

In [ ]:
tempos_lista = np.array([
    [0, 21],
    [21, 73],
    [73, 107],
    [107, 148],
    [148, 245],
    [245, 311]
])

betas_lista = []

for i in tempos_lista:
    betas_lista.append([0, 5])

betas_lista = np.array(betas_lista)

quantidade_de_betas = len(betas_lista)
concatenado = np.concatenate((betas_lista, tempos_lista), axis=0)

In [ ]:

# define beta em função do tempo
def define_beta(t, args):
    betas = args[:(quantidade_de_betas)]
    tempos = args[(quantidade_de_betas):]
    valor_para_retorno = 0

    for index, tempo in enumerate(tempos):
        if index == 0 and t <= tempo:
                valor_para_retorno = betas[index] 
        elif index > 0 and index <= len(tempos)-1 and t > tempos[index-1] and t < tempo:
                valor_para_retorno = betas[index]
        elif index == len(tempos)-1 and t >= tempo:
                valor_para_retorno = betas[index]


    return valor_para_retorno


In [ ]:

# condições iniciais

D0          = 0
N0          = 14136417
R0          = 0
H0          = 0
HV0         = 0
U0          = 0
UV0         = 0
V0          = 0
Rv0         = 0
Is0         = 2.015439771376298e-06
Ia0         = 1.8028646508967777e-06
Iav0        = 1.8028646508967777e-06 
Isv0        = 1.8028646508967777e-06 
E0          = 1.7639153732952095e-06
Ev0         = 1.7639153732952095e-06
S0          = (1-Is0-Ia0-E0)
Sv0         = 0 
Nw0         = 0
NwV0        = 0

condicoes_iniciais = [S0,E0,V0, Sv0, Ev0, Ia0, Is0, Iav0, Isv0, H0, HV0, U0, UV0, R0, Rv0, D0, Nw0, NwV0, 0, 0, 0]

In [ ]:
# Modelo com leaky
def SEIIHURD(t, y, args):
    
    beta, h, delta = define_beta(t, args)

    beta_v = beta

    tau = novos_vacinados[int(t)]

    delta_av = 0.31
    delta_sv = 0.31
    phi_e = 0.7
    k_v=1/3
    p_v = 0.2
    gamma_av = 1/3.5
    gamma_sv = 1/4
    gamma_vu=0.26
    qsi_v = 0.99
    eps=.75
    mi_vh=0.03
    mi_vu=0.08
    gamma_vh=0.36
    h_v = (1/5)*h

    psi = Psi[int(t)]
    S=y[0]
    E=y[1]
    V=y[2]
    Sv=y[3]
    Ev=y[4]
    Ia=y[5]
    Is=y[6]
    Iav=y[7]
    Isv=y[8]
    H=y[9]
    Hv=y[10]
    U=y[11]
    Uv=y[12]
    R=y[13]
    Rv=y[14]
    D=y[15]
    Nw=y[16]
    NwV=y[17]
    NHn=y[18]
    NHnV=y[19]
    NIAv=y[20]

    dSdt = (-beta*S*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) - tau/N
    dEdt = (beta*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv))*(S) - k*E
    dVdt = tau/N - phi_e*V
    dSvdt =  phi_e*V - ((1-eps)*beta_v*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv))
    dEvdt = ((1-eps)*beta_v*Sv*(Is+delta*Ia+delta_av*Iav+delta_sv*Isv)) - k_v*Ev
    
    dIadt = (1-p)*k*E - gamma_a*Ia
    dIsdt = p*k*E - gamma_s*Is
    dIavdt = (1-p_v)*k_v*Ev - gamma_av*Iav
    dIsvdt = p_v*k_v*Ev - gamma_sv*Isv
    dHdt = h*qsi*gamma_s*Is + (1-mi_u+ome_u*mi_u)*gamma_u*U - gamma_h*H
    dHvdt = h_v*qsi_v*gamma_sv*Isv + (1-mi_vu+ome_u*mi_vu)*gamma_vu*Uv - gamma_vh*Hv
    dUdt = h*(1-qsi)*gamma_s*Is + ome_h*gamma_h*H - gamma_u*U
    dUvdt = h_v*(1-qsi_v)*gamma_sv*Isv + ome_h*gamma_vh*Hv - gamma_u*Uv
    dRdt = gamma_a*Ia + (1-h)*gamma_s*Is + (1-mi_h)*(1-ome_h)*gamma_h*H
    dRvdt = gamma_av*Iav + (1-h_v)*gamma_sv*Isv + (1-mi_vh)*(1-ome_h)*(gamma_vh*Hv)
    dDdt = (1-ome_h)*(mi_h*gamma_h*H +mi_vh*gamma_vh*Hv) + (1-ome_u)*(mi_u*gamma_u*U+mi_vu*gamma_vu*Uv)
    dNwdt = p*k*E
    dNwVdt = tau/N
    dHndt = h*qsi*gamma_s*Is
    dHnvdt = h_v*qsi_v*gamma_sv*Isv
    dNIAvdt = p_v*k_v*Ev

    return [dSdt, dEdt, dVdt, dSvdt, dEvdt, dIadt, dIsdt, dIavdt, dIsvdt, dHdt, dHvdt, dUdt, dUvdt, dRdt, 
            dRvdt, dDdt, dNwdt, dNwVdt, dHndt, dHnvdt, dNIAvdt]

In [ ]:
def model(theta):
    sol = solve_ivp(fun = SEIIHURD, t_span = [0, len(novos_vacinados)-2], args=(theta,),y0 = condicoes_iniciais, t_eval=np.arange(len(novos_vacinados)-2))
    novos_casos = np.diff(sol.y[-5,:]).astype(float)
    novos_mortos = np.diff(sol.y[-6,:]).astype(float)
    return (np.r_[(novos_casos*N), (novos_mortos*N)], len(novos_casos), len(novos_mortos))

def lnlike(theta):
    model_dados, num_casos, num_mortes = model(theta)
    y1 = np.r_[np.diff(dados_casos)[:num_casos], np.diff(dados_mortes)[:num_mortes]]
    return -0.5 * np.sum(((y1 - model_dados)/0.05) ** 2)


In [ ]:
def check_limit_variable(guess, limits):
    final_value = 0
    for i in range(quantidade_de_betas):
        if guess < limits[0] or guess >= limits[1]:
            final_value = -np.inf
        
    return final_value

def check_limit_for_betas(guess):
    return check_limit_variable(guess, betas_lista[0])

def check_limit_for_times(guess, index_relative_to_list):
    return check_limit_variable(guess, tempos_lista[index_relative_to_list])

In [ ]:
def lnprior(theta):
    verification_list = []
    for i in range(quantidade_de_betas):
        verification_list.append(check_limit_for_betas(theta[i]))

    for j in np.arange(quantidade_de_betas, len(theta)):
        verification_list.append(check_limit_for_times(theta[j], (j-quantidade_de_betas)))
    

    return -np.inf if any(item != 0 for item in verification_list) else 0

def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf

    likelihood = lp + lnlike(theta)
    return likelihood

def main(p0,nwalkers,niter,ndim):
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

    print("Running burn-in...")
    p0, _, _ = sampler.run_mcmc(p0, 100)
    sampler.reset()

    print("Running production...")
    pos, prob, state = sampler.run_mcmc(p0, niter,progress=True)

    return sampler, pos, prob, state

In [ ]:
nwalkers = 200; niter = 100
intervals= np.array(concatenado)
par0 = np.random.rand(len(intervals))
initial = intervals[:,0] + par0 * (intervals[:,1] - intervals[:,0])
ndim = len(initial)
p0 = [np.array(initial) + 1e-7 * np.random.randn(ndim) for i in range(nwalkers)]
sampler1, pos1, prob1, state1 = main(p0,nwalkers,niter,ndim)
samples1 = sampler1.flatchain
theta_max1  = samples1[np.argmax(sampler1.flatlnprobability)]


In [ ]:
print(theta_max1)


In [ ]:
sol1 = solve_ivp(fun = SEIIHURD, t_span = [0, len(novos_vacinados)-2], args=(theta_max1,),y0=condicoes_iniciais, t_eval=np.arange(len(novos_vacinados)-2))

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-5,:] #mortes acumulados
#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_casos), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(np.diff(N*qi),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Casos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)
# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)
#Show Graph       
plt.show()

In [ ]:
# plotando mortes
ts0 =  np.arange(len(novos_vacinados)-1)

qi = sol1.y[-6,:] #mortes acumulados

#Criando plot
fig, ax = plt.subplots(figsize=(16,8))

#Desenhando grid no plot
ax.grid(which='major', axis='both', color='black',linewidth=1.,alpha=0.3)
ax.autoscale()

x_para_escala = np.arange(0, len(dados_datas)-1, 1)

ax.scatter(x_para_escala,  np.diff(dados_mortes), color='black',zorder=3,label='Reported data')
#Plot the fitted function as a line.
ax.plot(np.diff(N*(qi)),color='blue',label='Fitted function')
#Set the labels
ax.set_ylabel('Mortos',fontsize=22)
ax.set_xlabel('Dias', fontsize=22)

# plt.vlines([ 333.15303923, 439.95810453], ymin=0, ymax=200)
#Set the title
#The size of the numbers on the axixis
ax.tick_params(labelsize=18)

#Limiting the ammount of dates on the X axixs
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
#Rotating the dates for better visualization
# plt.setp(ax.get_xticklabels(), rotation=30)

#Show Graph       
plt.show()